## Imports

In [1]:
import sys
import os
import random
import pandas as pd
import pickle
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
import h5py
from h5py import File as HDF5File

import enum
print(enum.__file__)

/usr/lib/python3.8/enum.py


In [3]:
# Oh boy this might take a while
import tensorflow as tf
from tensorflow import keras

from keras.layers import Lambda, Input
from keras.layers import Dropout, Flatten, Dense
import keras.backend as K
from keras.models import Sequential, Model 
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import concatenate

2024-01-16 05:35:23.353458: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-16 05:35:24.013049: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import yaml
with open("../config.yaml") as fin:
    config = yaml.safe_load(fin)
    data_dir = config["data_dir"]
    model_dir = config["model_dir"]
    output_dir = config["output_dir"]

In [5]:
task_name = "scalar1"

## Loading data

See `cnn/data.py`.

In [6]:
with open(f"{data_dir}/processed/cnn/{task_name}_X_train.pkl", "rb") as fin:
    X_train = pickle.load(fin)
with open(f"{data_dir}/processed/cnn/{task_name}_Y_train.pkl", "rb") as fin:
    Y_train = pickle.load(fin)
with open(f"{data_dir}/processed/cnn/{task_name}_X_test.pkl", "rb") as fin:
    X_test = pickle.load(fin)
with open(f"{data_dir}/processed/cnn/{task_name}_Y_test.pkl", "rb") as fin:
    Y_test = pickle.load(fin)

In [7]:
print("Training shapes:")
print("  inputs:", X_train[0].shape, X_train[1].shape)
print("  labels:", Y_train.shape)

Training shapes:
  inputs: (21000, 4, 16, 1) (21000, 4, 128, 1)
  labels: (21000,)


## Build the model

In [8]:
visible1 = Input(shape=(4, 16, 1))
conv11 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible1)
pool11 = MaxPooling2D(pool_size=(2, 2))(conv11)
conv12 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool11)
pool12 = MaxPooling2D(pool_size=(2, 2))(conv12)
flat1 = Flatten()(pool12)
print ('cov11.shape', conv11.shape)
print ('pool1.shape', pool11.shape)
print ('cov12.shape', conv12.shape)
print ('pool2.shape', pool12.shape)

cov11.shape (None, 4, 16, 32)
pool1.shape (None, 2, 8, 32)
cov12.shape (None, 2, 8, 16)
pool2.shape (None, 1, 4, 16)


2024-01-16 05:35:25.849965: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9804 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:89:00.0, compute capability: 7.5
2024-01-16 05:35:25.851970: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9804 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:b1:00.0, compute capability: 7.5
2024-01-16 05:35:25.853730: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 9804 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:b2:00.0, compute capability: 7.5


In [9]:
visible2 = Input(shape=(4, 128, 1))
conv21 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible2)
pool21 = MaxPooling2D(pool_size=(2, 2))(conv21)
conv22 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool21)
pool22 = MaxPooling2D(pool_size=(2, 2))(conv22)
flat2 = Flatten()(pool22)
print ('cov21.shape', conv21.shape)
print ('poo21.shape', pool21.shape)
print ('cov22.shape', conv22.shape)
print ('poo22.shape', pool22.shape)

cov21.shape (None, 4, 128, 32)
poo21.shape (None, 2, 64, 32)
cov22.shape (None, 2, 64, 16)
poo22.shape (None, 1, 32, 16)


In [10]:
visible3 = Input(shape=(16, 16, 1))
conv31 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible3)
pool31 = MaxPooling2D(pool_size=(2, 2))(conv31)
conv32 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool31)
pool32 = MaxPooling2D(pool_size=(2, 2))(conv32)
flat3 = Flatten()(pool32)
print ('cov31.shape', conv31.shape)
print ('poo31.shape', pool31.shape)
print ('cov32.shape', conv32.shape)
print ('poo32.shape', pool32.shape)

cov31.shape (None, 16, 16, 32)
poo31.shape (None, 8, 8, 32)
cov32.shape (None, 8, 8, 16)
poo32.shape (None, 4, 4, 16)


In [11]:
visible4 = Input(shape=(16, 8, 1))
conv41 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible4)
pool41 = MaxPooling2D(pool_size=(2, 2))(conv41)
conv42 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool41)
pool42 = MaxPooling2D(pool_size=(2, 2))(conv42)
flat4 = Flatten()(pool32)
print ('cov41.shape', conv41.shape)
print ('poo41.shape', pool41.shape)
print ('cov42.shape', conv42.shape)
print ('poo42.shape', pool42.shape)

cov41.shape (None, 16, 8, 32)
poo41.shape (None, 8, 4, 32)
cov42.shape (None, 8, 4, 16)
poo42.shape (None, 4, 2, 16)


In [12]:
merge = concatenate([flat1, flat2, flat3, flat4])

In [13]:
# interpretation model
hidden1 = Dense(20, activation='relu')(merge)
hidden2 = Dense(20, activation='relu')(hidden1)
output = Dense(3)(hidden2)
cnn = Model(inputs=[visible1, visible2, visible3, visible4], outputs=output)

In [14]:
# summarize layers
# print(cnn.summary())

In [15]:
cnn.inputs

[<KerasTensor: shape=(None, 4, 16, 1) dtype=float32 (created by layer 'input_1')>,
 <KerasTensor: shape=(None, 4, 128, 1) dtype=float32 (created by layer 'input_2')>,
 <KerasTensor: shape=(None, 16, 16, 1) dtype=float32 (created by layer 'input_3')>,
 <KerasTensor: shape=(None, 16, 8, 1) dtype=float32 (created by layer 'input_4')>]

## Train the model

In [16]:
def train_iteration(lr, epochs):
    print(f"=== Training with lr={lr} for {epochs} epochs ===")
    cnn.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"]
    )
    return cnn.fit(
        X_train, Y_train,
        epochs=epochs, batch_size=128,
        validation_data=(X_test, Y_test),
    )

In [17]:
train_iteration(lr=1e-2, epochs=5)

=== Training with lr=0.01 for 5 epochs ===
Epoch 1/5


2024-01-16 05:35:30.935574: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8700
2024-01-16 05:35:31.113067: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-16 05:35:31.471815: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f5c5dcc86b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-16 05:35:31.471900: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-01-16 05:35:31.471929: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-01-16 05:35:31.471954: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (2): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-01-16 05:35:31.479375: I tensorflow/compiler/mlir/tensorflow/utils/du

165/165 [==============================] - 8s 17ms/step - loss: 86.4752 - accuracy: 0.7807 - val_loss: 0.1879 - val_accuracy: 0.9421
Epoch 2/5
165/165 [==============================] - 1s 8ms/step - loss: 0.1277 - accuracy: 0.9600 - val_loss: 0.1378 - val_accuracy: 0.9590
Epoch 3/5
165/165 [==============================] - 1s 6ms/step - loss: 0.0928 - accuracy: 0.9709 - val_loss: 0.0827 - val_accuracy: 0.9737
Epoch 4/5
165/165 [==============================] - 1s 7ms/step - loss: 0.0801 - accuracy: 0.9737 - val_loss: 0.2748 - val_accuracy: 0.8833
Epoch 5/5
165/165 [==============================] - 1s 8ms/step - loss: 0.0952 - accuracy: 0.9707 - val_loss: 0.0941 - val_accuracy: 0.9704


In [18]:
train_iteration(lr=2e-3, epochs=5)

=== Training with lr=0.002 for 5 epochs ===
Epoch 1/5
165/165 [==============================] - 5s 13ms/step - loss: 0.2273 - accuracy: 0.9471 - val_loss: 0.1085 - val_accuracy: 0.9658
Epoch 2/5
165/165 [==============================] - 2s 10ms/step - loss: 0.0825 - accuracy: 0.9725 - val_loss: 0.0930 - val_accuracy: 0.9725
Epoch 3/5
165/165 [==============================] - 2s 10ms/step - loss: 0.0769 - accuracy: 0.9754 - val_loss: 0.0918 - val_accuracy: 0.9718
Epoch 4/5
165/165 [==============================] - 2s 10ms/step - loss: 0.0701 - accuracy: 0.9773 - val_loss: 0.1127 - val_accuracy: 0.9681
Epoch 5/5
165/165 [==============================] - 2s 11ms/step - loss: 0.0737 - accuracy: 0.9756 - val_loss: 0.0829 - val_accuracy: 0.9748


In [19]:
train_iteration(lr=1e-3, epochs=5)

=== Training with lr=0.001 for 5 epochs ===
Epoch 1/5
165/165 [==============================] - 5s 16ms/step - loss: 0.0598 - accuracy: 0.9819 - val_loss: 0.0753 - val_accuracy: 0.9778
Epoch 2/5
165/165 [==============================] - 2s 9ms/step - loss: 0.0493 - accuracy: 0.9835 - val_loss: 0.0635 - val_accuracy: 0.9802
Epoch 3/5
165/165 [==============================] - 2s 10ms/step - loss: 0.0531 - accuracy: 0.9836 - val_loss: 0.0681 - val_accuracy: 0.9795
Epoch 4/5
165/165 [==============================] - 2s 11ms/step - loss: 0.0489 - accuracy: 0.9846 - val_loss: 0.0650 - val_accuracy: 0.9793
Epoch 5/5
165/165 [==============================] - 2s 11ms/step - loss: 0.0430 - accuracy: 0.9865 - val_loss: 0.0639 - val_accuracy: 0.9819


In [20]:
train_iteration(lr=2e-4, epochs=5)

=== Training with lr=0.0002 for 5 epochs ===
Epoch 1/5
165/165 [==============================] - 5s 14ms/step - loss: 0.0321 - accuracy: 0.9904 - val_loss: 0.0562 - val_accuracy: 0.9836
Epoch 2/5
165/165 [==============================] - 2s 10ms/step - loss: 0.0297 - accuracy: 0.9912 - val_loss: 0.0568 - val_accuracy: 0.9833
Epoch 3/5
165/165 [==============================] - 2s 10ms/step - loss: 0.0280 - accuracy: 0.9918 - val_loss: 0.0525 - val_accuracy: 0.9846
Epoch 4/5
165/165 [==============================] - 2s 11ms/step - loss: 0.0269 - accuracy: 0.9919 - val_loss: 0.0599 - val_accuracy: 0.9830
Epoch 5/5
165/165 [==============================] - 2s 11ms/step - loss: 0.0264 - accuracy: 0.9922 - val_loss: 0.0532 - val_accuracy: 0.9850


In [21]:
train_iteration(lr=2e-5, epochs=30)

=== Training with lr=2e-05 for 30 epochs ===
Epoch 1/30
165/165 [==============================] - 5s 16ms/step - loss: 0.0232 - accuracy: 0.9934 - val_loss: 0.0524 - val_accuracy: 0.9853
Epoch 2/30
165/165 [==============================] - 2s 10ms/step - loss: 0.0227 - accuracy: 0.9935 - val_loss: 0.0523 - val_accuracy: 0.9854
Epoch 3/30
165/165 [==============================] - 2s 11ms/step - loss: 0.0224 - accuracy: 0.9935 - val_loss: 0.0527 - val_accuracy: 0.9853
Epoch 4/30
165/165 [==============================] - 2s 10ms/step - loss: 0.0223 - accuracy: 0.9933 - val_loss: 0.0525 - val_accuracy: 0.9856
Epoch 5/30
165/165 [==============================] - 2s 10ms/step - loss: 0.0222 - accuracy: 0.9936 - val_loss: 0.0528 - val_accuracy: 0.9852
Epoch 6/30
165/165 [==============================] - 2s 11ms/step - loss: 0.0220 - accuracy: 0.9937 - val_loss: 0.0526 - val_accuracy: 0.9854
Epoch 7/30
165/165 [==============================] - 2s 11ms/step - loss: 0.0220 - accuracy: 0.9

In [22]:
train_iteration(lr=1e-5, epochs=10)

=== Training with lr=1e-05 for 10 epochs ===
Epoch 1/10
165/165 [==============================] - 5s 14ms/step - loss: 0.0191 - accuracy: 0.9947 - val_loss: 0.0536 - val_accuracy: 0.9850
Epoch 2/10
165/165 [==============================] - 2s 10ms/step - loss: 0.0191 - accuracy: 0.9947 - val_loss: 0.0537 - val_accuracy: 0.9852
Epoch 3/10
165/165 [==============================] - 1s 9ms/step - loss: 0.0191 - accuracy: 0.9947 - val_loss: 0.0541 - val_accuracy: 0.9853
Epoch 4/10
165/165 [==============================] - 2s 10ms/step - loss: 0.0190 - accuracy: 0.9949 - val_loss: 0.0538 - val_accuracy: 0.9853
Epoch 5/10
165/165 [==============================] - 1s 8ms/step - loss: 0.0189 - accuracy: 0.9948 - val_loss: 0.0536 - val_accuracy: 0.9852
Epoch 6/10
165/165 [==============================] - 1s 7ms/step - loss: 0.0189 - accuracy: 0.9948 - val_loss: 0.0540 - val_accuracy: 0.9852
Epoch 7/10
165/165 [==============================] - 1s 6ms/step - loss: 0.0188 - accuracy: 0.9949 

In [ ]:
# Save this model
os.makedirs(f"{model_dir}", exist_ok=True)

print(f"task name: {task_name}")
cnn.save(f"{model_dir}/{task_name}_cnn")